In [20]:
import sys
sys.path.append('../')

In [2]:
import lemur
x = lemur.tensor([1.0], requires_grad=True)
y = lemur.tensor([4.0], requires_grad=True)

z = x + y  
w = z * x  
w = w.relu() 

w.backward() 
print("Result from main:", w)

Result from main: tensor: 0x156a0f1b0
  requires_grad = true
  k:
  kernel_tensor: 0x156aa0f80
    length     = 1
    shape      = [1, 1, 1, 1, 1]
    stride     = [1, 1, 1, 1, 1]
    computed   = false
    data:
      tensor([[[[[5.0000]]]]])
  grad:
  [NULL kernel_tensor]



In [31]:
import lemur

In [36]:
shape = [1,1,1,1,10]

In [37]:
%%timeit
x = lemur.LemurTensor(shape=shape, requires_grad=False)
y = lemur.LemurTensor(shape=shape, requires_grad=False)
x + y  

5.43 µs ± 781 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [38]:
import torch 

In [39]:
%%timeit
x = torch.empty(shape)
y = torch.empty(shape)
x + y  

2.81 µs ± 102 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [11]:
t0 = torch.tensor([10.], requires_grad=True)
t1 = torch.tensor([12.], requires_grad=True)
t2 = torch.tensor([-15.], requires_grad=True)
t4 = t0 * t1 + t2
t4


tensor([105.], grad_fn=<AddBackward0>)

In [12]:
t4.backward()

In [13]:
t0.grad, t1.grad, t2.grad

(tensor([12.]), tensor([10.]), tensor([1.]))

In [25]:
a = lemur.tensor([[10.,12.],[-1.,-4.]], requires_grad = True)
b = lemur.rand_like(a)
c = (a @ b).mean()
c.backward()
a.grad

tensor([[0.4100, 0.2642],
        [0.4100, 0.2642]])

In [26]:
c

tensor(5.8041, grad_fn=<MeanBackward0>)

In [ ]:
a = lemur.tensor([...], requires_grad = True)
b = lemur.tensor([...], requires_grad = True)
c = (a + b).relu()

In [31]:
x = torch.tensor([[1], [2], [3]])
x.size(), x.stride()

(torch.Size([3, 1]), (1, 1))

In [35]:
x.expand(3, 4)


tensor([[1, 1, 1, 1],
        [2, 2, 2, 2],
        [3, 3, 3, 3]])

In [47]:
x.retain_grad()
x.grad

In [62]:
x = torch.tensor([[1],[2]]).expand((2,10))
x.shape, x.stride()

(torch.Size([2, 10]), (1, 0))

In [64]:
y = x.reshape(20)

In [67]:
y.shape, y.stride

(torch.Size([20]), <function Tensor.stride>)

In [86]:
x = torch.tensor([[10.]]).expand(1,10)
x.requires_grad = True
x.stride()

(1, 0)

In [88]:
x.sum().backward()
x.grad.stride()

(10, 1)

In [99]:
x = torch.rand((3,4)).t() #(1,4)
x.requires_grad = True


In [100]:
x.sum().backward()

In [101]:
x.grad.stride()

(1, 4)

In [83]:
float arr[27]
shape (3,3,3) 
stride (9,3,1)

In [ ]:
t0 -> 1,1 
   -> length = 1

t1 -> 1,10 
   -> length = 1   
   -> comesfrom -> t0, expand, 1 10






In [73]:
x.stride()

(1, 0)

In [102]:
import torch

In [122]:
x = torch.randn((10,10), requires_grad=True)

In [124]:
x[x>0]

tensor([0.9947, 0.2142, 0.6538, 0.6045, 0.8833, 0.3018, 1.1432, 0.9210, 0.4568,
        0.5722, 1.0691, 0.3217, 0.0436, 0.0290, 0.5943, 1.7333, 0.3016, 0.1453,
        0.1869, 0.5580, 1.3963, 0.6988, 0.1913, 0.2689, 0.3597, 2.0577, 1.2033,
        0.1872, 0.2806, 0.0055, 0.6726, 0.1652, 0.9872, 0.0903, 0.2512, 1.3771,
        0.1913, 0.3629, 1.8730, 0.2753, 0.0473, 0.2828, 0.0151, 1.0343, 0.6497,
        0.2800, 3.5641, 0.3396, 0.9233], grad_fn=<IndexBackward0>)

In [117]:
x.backward()